In [1]:
import numpy as np
from random import shuffle
import scipy.io.wavfile
import MFCC
import gmmhmm

import glob
import pickle

In [2]:
A = np.array([[.65, .35], [.15, .85]])
pi = np.array([.8, .2])
weights = np.array([[.7, .2, .1], [.1, .5, .4]])
means1 = np.array([[0., 17., -4.], [5., -12., -8.], [-16., 22., 2.]])
means2 = np.array([[-5., 3., 23.], [-12., -2., 14.], [15., -32., 0.]])
means = np.array([means1, means2])
covars1 = np.array([5*np.eye(3), 7*np.eye(3), np.eye(3)])
covars2 = np.array([10*np.eye(3), 3*np.eye(3), 4*np.eye(3)])
covars = np.array([covars1, covars2])
Gmmhmm = [A, weights, means, covars, pi]
# using 1 as current state
sample_component = np.argmax(np.random.multinomial(1, weights[1,:]))
sample = np.random.multivariate_normal(means[1, sample_component, :], -covars[1, sample_component, :, :])

### Problem 1

Write a function which accepts a GMMHMM in the format above as well as an integer n_sim, and which simulates the GMMHMM process, generating n_sim different observations. Do so by implementing the following function declaration.

In [3]:
def sample_gmmhmm(gmmhmm, n_sim):
    """
    Simulate sampling from a GMMHMM.
    Returns
    -------
    sequence of states : ndarray of shape (n_sim,)
    
    observations : ndarray of shape (n_sim, K) ;(column vectors of length K)
    """
    [A, weights, means, covars, pi] = gmmhmm
    
    #classes = len(pi)
    
    state1 = np.argmax(np.random.multinomial(1, pi))
    
    obs = []
    st = [state1]
    
    for _ in range(n_sim):
     
        sample_component = np.argmax(np.random.multinomial(1, weights[state1,:]))
        sample = np.random.multivariate_normal(means[state1, sample_component, :], -covars[state1, sample_component, :, :])
        obs.append(sample)
        
        next_state = np.argmax(np.random.multinomial(1, A[state1]))
        st.append(next_state)
        
        state1 = next_state

        
    return st, obs

In [4]:
print(sample_gmmhmm(Gmmhmm, 3))

([0, 1, 1, 1], [array([-16.,  22.,   2.]), array([-12.,  -2.,  14.]), array([ 15., -32.,   0.])])


### Problem 2

Obtain 30 (or more) recordings for each of the words/phrases mathematics, biology, political science, psychology, and statistics. These audio samples should be 2 seconds in duration, recorded at a rate of 44100 samples per second, with samples stored as 16-bit signed integers in WAV format. Load the recordings into Python using scipy.io.wavfile.read.

If the audio files have two channels, average these channels to obtain an array of length 88200 for each sample. Extract the MFCCs from each sample using code from the file MFCC.py. Store the MFCCs for each word in a separate list. You should have five lists, each containing 50 MFCC arrays, corresponding to each of the five words under consideration.


In [28]:
names = ["biology", "mathematics", "polysci", "psychology", "statistics"]

all_files = []

for n in names:
    
    # get strings of file names
    files = glob.glob("CDHMMSoundFiles/"+n+'/'+n+'*')
    
    sounds = []
    for f in files:
        rate, signal = scipy.io.wavfile.read(f)
        if rate != 44100: # check that all are 1 by 88200
            raise ValueError("Bad sample rate")
        
        sounds.append(MFCC.extract(signal))
        
    all_files.append(sounds)
    

['CDHMMSoundFiles/biology/biology_01.wav', 'CDHMMSoundFiles/biology/biology_02.wav', 'CDHMMSoundFiles/biology/biology_03.wav', 'CDHMMSoundFiles/biology/biology_04.wav', 'CDHMMSoundFiles/biology/biology_05.wav', 'CDHMMSoundFiles/biology/biology_06.wav', 'CDHMMSoundFiles/biology/biology_07.wav', 'CDHMMSoundFiles/biology/biology_08.wav', 'CDHMMSoundFiles/biology/biology_09.wav', 'CDHMMSoundFiles/biology/biology_10.wav', 'CDHMMSoundFiles/biology/biology_11.wav', 'CDHMMSoundFiles/biology/biology_12.wav', 'CDHMMSoundFiles/biology/biology_13.wav', 'CDHMMSoundFiles/biology/biology_14.wav', 'CDHMMSoundFiles/biology/biology_15.wav', 'CDHMMSoundFiles/biology/biology_16.wav', 'CDHMMSoundFiles/biology/biology_17.wav', 'CDHMMSoundFiles/biology/biology_18.wav', 'CDHMMSoundFiles/biology/biology_19.wav', 'CDHMMSoundFiles/biology/biology_20.wav', 'CDHMMSoundFiles/biology/biology_21.wav', 'CDHMMSoundFiles/biology/biology_22.wav', 'CDHMMSoundFiles/biology/biology_23.wav', 'CDHMMSoundFiles/biology/biology_

In [27]:
#print(len(all_files)) # 5

# Testing
#print(glob.glob("CDHMMSoundFiles/biology/biology*"))

B, M, Po, Ps, S = all_files

B = B[:30]
M = M[:30]
Po = Po[:30]
Ps = Ps[:30]
S = S[:30]

print(np.shape(B))
print(np.shape(M))
print(np.shape(Po))
print(np.shape(Ps))
print(np.shape(S)) # inside ones are different sizes

(29,)
(30, 198, 10)
(30, 198, 10)
(30, 198, 10)
(30, 198, 10)
(29,)


In [7]:
def initialize(n):
    '''
    n: number of states (integer)
    Returns
    pi: random initial state distribution 
    T: transition matrix (row-stochastic)
    '''
    
    pi = np.random.normal(1/n, .01, size=n)
    pi[-1] = 1- np.sum(pi[:-1])

    T = np.random.normal(1/n, .01, size=(n,n))
    T[:,-1] = 1- np.sum(T[:, :-1], axis=1)
    
    return pi, T

In [ ]:
#Pi, T, = initialize(5)
#print(T)
#print(np.sum(T, axis=1))

### Problem 3

Partition each list of MFCCs into a training set of 20 samples, and a test set of the remaining 10 samples.
Using the training sets, train a GMMHMM on each of the words from the previous problem with at least 10 random restarts, keeping the best model for each word (the one with the highest log-likelihood). This process may take several minutes. Since you will not want to run this more than once, you will want to save the best model for each word to disk using the pickle module so that you can use it later.

In [9]:
# split into test and train data

# Biology
shuffle(B)
B_train, B_test = B[:20], B[20:]
# Mathematics
shuffle(M)
M_train, M_test = M[:20], M[20:]
# Polysci
shuffle(Po)
Po_train, Po_test = Po[:20], Po[20:]
# Psychology
shuffle(Ps)
Ps_train, Ps_test = Ps[:20], Ps[20:]
# Statistics
shuffle(S)
S_train, S_test = S[:20], S[20:]


In [14]:
# Biology model
#logprob = -np.inf
#for _ in range(10):
    
#    startprob, transmat = initialize(5)     # startprob is pi, transmat is T

#    modelB = gmmhmm.GMMHMM(n_components=5, n_mix=3, transmat=transmat,
#                          startprob=startprob, cvtype='diag')

#    modelB.covars_prior = 0.01
#    modelB.fit(B_train, init_params='mc', var=0.1)

#    if modelB.logprob > logprob:
#        logprob = modelB.logprob
    
#        pickle.dump(modelB, open("best_modelB.pickle", "wb"))

        
# Mathematics model
#logprob = -np.inf
#for _ in range(10):
    
#    pi, T = initialize(5)

#    modelM = gmmhmm.GMMHMM(n_components=5, n_mix=3, transmat=T,
#                          startprob=pi, cvtype='diag')

#    modelM.covars_prior = 0.01
#    modelM.fit(M_train, init_params='mc', var=0.1)

#    if modelM.logprob > logprob:
#        logprob = modelM.logprob
    
#        pickle.dump(modelM, open("best_modelM.pickle", "wb"))

     
# polysci model
#logprob = -np.inf
#for _ in range(10):
    
#    pi, T = initialize(5)

#    modelPo = gmmhmm.GMMHMM(n_components=5, n_mix=3, transmat=T,
#                          startprob=pi, cvtype='diag')

#    modelPo.covars_prior = 0.01
#    modelPo.fit(Po_train, init_params='mc', var=0.1)

#    if modelPo.logprob > logprob:
#        logprob = modelPo.logprob
    
#        pickle.dump(modelPo, open("best_modelPo.pickle", "wb"))


# psychology model
#logprob = -np.inf
#for _ in range(10):
    
#    pi, T = initialize(5)

#    modelPs = gmmhmm.GMMHMM(n_components=5, n_mix=3, transmat=T,
#                          startprob=pi, cvtype='diag')

#    modelPs.covars_prior = 0.01
#    modelPs.fit(Ps_train, init_params='mc', var=0.1)

#    if modelPs.logprob > logprob:
#        logprob = modelPs.logprob
    
#        pickle.dump(modelPs, open("best_modelPs.pickle", "wb"))


# statistics model
#logprob = -np.inf
#for _ in range(10):   
#    pi, T = initialize(5)
#    modelS = gmmhmm.GMMHMM(n_components=5, n_mix=3, transmat=T,
#                          startprob=pi, cvtype='diag')
#    modelS.covars_prior = 0.01
#    modelS.fit(S_train, init_params='mc', var=0.1)
#    if modelS.logprob > logprob:
#        logprob = modelS.logprob   
#        pickle.dump(modelS, open("best_modelS.pickle", "wb"))


In [15]:
modelB = pickle.load( open("best_ModelB.pickle", "rb") )
modelM = pickle.load( open("best_ModelM.pickle", "rb") )
modelPo = pickle.load( open("best_ModelPo.pickle", "rb") )
modelPs = pickle.load( open("best_ModelPs.pickle", "rb") )
modelS = pickle.load( open("best_ModelS.pickle", "rb") )

### Problem 4

Classify the 10 test samples for each word. How does your system perform? Which words are the hardest to correctly classify? Make a dictionary containing the accuracy of the classification of your five testing sets. Specifically, the words/phrases will be the keys, and the values will be the percent accuracy.

In [29]:
# classify my 10 'test' samples for each word

my_dict ={0:'biology', 1:'mathematics', 2:'political science',
        3:'psychology', 4:'statistics'}


print(len(B_test), len(M_test), len(Po_test), len(Ps_test), len(S_test))

acc_pct = np.zeros(5)

i = 0
for test in [B_test, M_test, Po_test, Ps_test, S_test]:
    
    for obs in test: 
        s1 = modelB.score(obs)
        s2 = modelM.score(obs)
        s3 = modelPo.score(obs)
        s4 = modelPs.score(obs)
        s5 = modelS.score(obs)
        # vote on best model
        choice = np.argmax([s1,s2,s3,s4,s5])

        if choice == i:
            acc_pct[i] += 1

        print("Classified {} as : {}".format(my_dict[i], my_dict[choice]))

    i+=1

10 10 10 10 9
Classified Biology as : statistics
Classified Biology as : biology
Classified Biology as : biology
Classified Biology as : biology
Classified Biology as : biology
Classified Biology as : biology
Classified Biology as : biology
Classified Biology as : biology
Classified Biology as : biology
Classified Biology as : biology
Classified Mathematics as : mathematics
Classified Mathematics as : mathematics
Classified Mathematics as : mathematics
Classified Mathematics as : mathematics
Classified Mathematics as : mathematics
Classified Mathematics as : mathematics
Classified Mathematics as : mathematics
Classified Mathematics as : mathematics
Classified Mathematics as : mathematics
Classified Mathematics as : mathematics
Classified Political Science as : political science
Classified Political Science as : political science
Classified Political Science as : political science
Classified Political Science as : political science
Classified Political Science as : political science
Cla

In [26]:
acc_pct /= 10. #*np.ones(5)  

#   {:.2f}    .format( )
    
words =['biology', 'mathematics', 'political science',
        'psychology', 'statistics']

word_accuracy = dict(zip(words, acc_pct))

print(word_accuracy)

[  9.  10.  10.   9.   9.]
{'biology': 0.90000000000000002, 'mathematics': 1.0, 'political science': 1.0, 'psychology': 0.90000000000000002, 'statistics': 0.90000000000000002}


In [ ]:
# TESTING
#a = ['a','b','c','d']
#b = [1,2,3,4]
#print(dict(zip(a,b)))
#print(dict(zip(b,a)))

# output
#{'a': 1, 'b': 2, 'c': 3, 'd': 4}
#{1: 'a', 2: 'b', 3: 'c', 4: 'd'}